<a href="https://colab.research.google.com/github/HeshamEL-Shreif/NER_spacy/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import spacy
from spacy.tokens import DocBin
from datasets import load_dataset
import transformers

In [ ]:
dataset = load_dataset("conll2003")
nlp = spacy.blank("en")
doc_bin = DocBin()

In [ ]:
dataset

In [ ]:
train_examples = dataset["train"]['tokens']
train_tags = dataset["train"]['ner_tags']

In [ ]:
print(train_examples[0])
print(train_tags[0])

In [ ]:
labels = dataset["train"].features["ner_tags"].feature.names

In [ ]:
for tokens, ner_tags in zip(dataset["train"]["tokens"], dataset["train"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

In [ ]:
for tokens, ner_tags in zip(dataset["validation"]["tokens"], dataset["validation"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("dev.spacy")

In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --training.max_epochs 10 --gpu-id 0

In [ ]:
for tokens, ner_tags in zip(dataset["test"]["tokens"], dataset["test"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("test.spacy")

In [ ]:
!python -m spacy evaluate output/model-best ./test.spacy --output metrics.json